# Investment model dispatch patterns
Routines for analyzing dispatch patterns of *pommesinvest* runs

## Package imports

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from pommesevaluation.investment_results_inspection import (
    preprocess_raw_results, aggregate_investment_results
)

## Parameters and workflow settings

In [ ]:
# Model configuration
time_frame_in_years = 31
frequency = "8H"
dr_scenario = "50"
dr_scenarios = ["none", "5", "50", "95"]

# Paths, filenames and color codes
path_results = "./model_results/"
path_processed_data = "./data_out/"
path_plots = "./plots/"

filename = (
    f"investment_LP_start-2020-01-01_{time_frame_in_years}"
    f"-years_simple_freq_{frequency}"
)
if dr_scenario != "none":
    file_add_on = f"_with_dr_{dr_scenario}_production"
else:
    file_add_on = "_no_dr_50_production"
file_extension = ".csv"

# color codes
FUELS_EXISTING = {
    "uranium": "#e50000",
    "lignite": "#7f2b0a",
    "hardcoal": "#000000",
    "mixedfuels": "#a57e52",
    "otherfossil": "#d8dcd6",
}

FUELS = {
    "biomass": "#15b01a",
    "hydrogen": "#6fa8dc",
    "natgas": "#ffd966",
    "oil": "#aaa662",
    "waste": "#c04e01"
}

# Workflow and output configuration
plt.rcParams.update({'font.size': 12})
rounding_precision = 2

# Single scenario analyses
Inspect the results for a single scenario model run.
## Read in, preprocess and aggregate data
* Use routine originally developped for investment model to preprocess raw results. Therefore, transpose back and forth.

In [ ]:
production_results_raw = pd.read_csv(f"{path_results}{filename}{file_add_on}{file_extension}", index_col=0).T
processed_results = preprocess_raw_results(
    production_results_raw, investments=False
).drop(columns="year").round(rounding_precision)
aggregated_results = aggregate_investment_results(
    processed_results, energy_carriers={**FUELS_EXISTING, **FUELS}, by="energy_carrier", investments=False
)

In [ ]:
aggregated_results